# ニューラルネットワークによるアヤメの分類(keras)

In [ ]:
import numpy as np
np.set_printoptions(suppress=True) 
np.random.seed(3)

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers

# 環境の都合上GPUを無効化
import keras
import tensorflow as tf
import keras.backend as K
K.tensorflow_backend.set_session(tf.Session(config=tf.ConfigProto(device_count = {'GPU': 0})))

log_dir = 'tflog'
import keras.callbacks
tb_cb = keras.callbacks.TensorBoard(log_dir=log_dir, 
histogram_freq=1, write_graph=True, write_images=True)


In [ ]:
# データの読込
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()

# 入力変数と説明変数の切り分け
X = iris.data.astype(np.float32)
Y = iris.target.flatten()

# 訓練データと検証データに分割(70%を訓練用に使用)
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, train_size=0.7, test_size=0.3)

In [ ]:
# モデルの構成
model = Sequential()
model.add(Dense(30, input_dim=4))
model.add(Activation('relu'))
model.add(Dense(20, input_dim=30))
model.add(Dense(3, input_dim=20))
model.add(Activation('softmax')) 
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer='sgd' ,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
epoch = 100
batchsize = 50
#model.fit(features, targets, nb_epoch=epoch, batch_size=batchsize)
result = model.fit(train_X, train_Y, epochs=epoch, batch_size=batchsize,validation_data=(train_X, train_Y), callbacks=[tb_cb])

In [ ]:
# モデル精度とロス率の表示(検証データ)
score = model.evaluate(test_X, test_Y, verbose=0)
print('Test accuracy:', score[1])
print('Test loss:', score[0])

In [ ]:
# モデル精度の可視化
%matplotlib inline
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15,5

plt.plot(range(1, epoch+1), result.history['acc'], label="training")
plt.plot(range(1, epoch+1), result.history['val_acc'], label="validation")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

plt.plot(range(1, epoch+1), result.history['loss'], label="training")
plt.plot(range(1, epoch+1), result.history['val_loss'], label="validation")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend()
plt.show()

In [ ]:
# 元データの150件目(X[149])でクラスごとの分類確率を表示[Setosa, Versicolor, Virginica]
x = np.expand_dims(X[ 149 ], axis=0)
pred = model.predict(x)
print(pred)

In [ ]:
# 元データの150件目(X[149])で分類結果を表示[0=Setosa,1=Versicolor, 2=Virginica]
x = np.expand_dims(X[ 149 ], axis=0)
pred = model.predict_classes(x)
print(pred)

In [ ]:
# 元データの150件目(Y[149])の正解を表示(0=Setosa,1=Versicolor, 2=Virginica)
print ([Y[149]])

In [ ]:
#  参考: 手動で値設定する方法
x = np.expand_dims([ 5.9, 3. , 5.1, 1.8 ], axis=0)
pred = model.predict_classes(x)
print(pred)

In [ ]:
# Jupyter上でカーネルリセット&GPUメモリ解放
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()